In [1]:
from bs4 import BeautifulSoup
import requests

Use BeautifulSoup retrive data from the web

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')
table = soup.find('table')

Find the table informaiton

In [3]:
data=[]
head = [items.text.strip() for items in table.find_all('th')]

for rows in table.find_all('tr'):
    cols = [items.text.strip() for items in rows.find_all('td')]
    data.append(cols)
    
data[0]=head
data[0:5]

[['Postcode', 'Borough', 'Neighbourhood'],
 ['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village']]

In [4]:
import numpy as np
import pandas as pd

In [5]:
# create the dataframe with retrived data
df=pd.DataFrame(data[1:], columns=data[0])
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [6]:
df.shape

(288, 3)

In [7]:
pc_canada = df.copy()
# remove rows with a borough that is Not assigned.
pc_canada=pc_canada[pc_canada['Borough']!='Not assigned']
pc_canada.sort_values(by = ['Postcode','Borough']).head()               

,Postcode,Borough,Neighbourhood
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern
27,M1C,Scarborough,Highland Creek
28,M1C,Scarborough,Rouge Hill
29,M1C,Scarborough,Port Union


In [8]:
pc_canada.shape

(211, 3)

In [9]:
# Combine rows with same Postcode and Borough
pc_canada_g = pc_canada.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: ','.join(x)).reset_index()

# If a cell has a borough but a Not assigned neighborhood, replace with the borough
pc_canada_g['Neighbourhood'] = np.where(pc_canada_g['Neighbourhood'] == 'Not assigned', pc_canada_g['Borough'], pc_canada_g['Neighbourhood'])

pc_canada_g.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [10]:
pc_canada_g.shape

(103, 3)